# Fraud Detection Analysis | Exploratory Data Analysis with SQL

In this analysis, we will load our fraud detection dataset into a SQL server to perform various queries and gain insights. We will start by importing the dataset into the server, ensuring that the data is properly structured and indexed for efficient querying. Once the data is loaded, we will execute a series of SQL queries to explore the dataset, identify patterns, and detect any anomalies. This will include aggregating data, filtering records, joining tables, and using advanced SQL functions to uncover trends and correlations that could indicate fraudulent activities. Through this process, we aim to build a comprehensive understanding of the dataset and lay the groundwork for further analysis and model building.

Load the SQL extension for Jupyter

In [2]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Establish SQLite connection

In [3]:
# Import necessary libraries
import csv, sqlite3
import pandas as pd

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

Connect to SQLite using SQL magic

In [4]:
%sql sqlite:///my_data1.db

Upload the dataset in the database; it's done through chunks because SQLite has a limit on the number of variables (columns) that can be used in a single SQL statement.

In [6]:
# Read CSV into pandas DataFrame
file_name = 'DATA/creditcard.csv'
data = pd.read_csv(file_name)

# Import DataFrame to SQLite table in chunks
chunk_size = 1000  # Adjust the chunk size if necessary
for i in range(0, len(data), chunk_size):
    data_chunk = data.iloc[i:i + chunk_size]
    data_chunk.to_sql("CREDIT_FRAUD", con, if_exists='append', index=False)

Create a table 

In [7]:
%sql create table CREDIT_FRAUD_TABLE as select * from CREDIT_FRAUD

 * sqlite:///my_data1.db
Done.


[]

Show the first 5 datapoints of the table (only Time and Amount)

In [41]:
import prettytable as pt

# Create a PrettyTable object and set the style
table = pt.PrettyTable()
table.set_style(pt.MSWORD_FRIENDLY)

# Fetch the results from the SQL query
results = cur.execute("SELECT Time, Amount FROM CREDIT_FRAUD_TABLE LIMIT 5").fetchall()

# Get the column names from the cursor
column_names = [description[0] for description in cur.description]

# Add the column names to the table
table.field_names = column_names

# Add the rows to the table
for row in results:
    table.add_row(row)

# Print the table
print(table)

| Time | Amount |
| 0.0  | 149.62 |
| 0.0  |  2.69  |
| 1.0  | 378.66 |
| 1.0  | 123.5  |
| 2.0  | 69.99  |


Show how many datapoints based on Class

In [25]:
# Create a PrettyTable object and set the style
table = pt.PrettyTable()
table.set_style(pt.MSWORD_FRIENDLY)

# Fetch the results from the SQL query to count distinct values and group by Class
results = cur.execute("SELECT Class, COUNT(*) AS Count FROM CREDIT_FRAUD_TABLE GROUP BY Class").fetchall()

# Get the column names from the cursor
column_names = [description[0] for description in cur.description]

# Add the column names to the table
table.field_names = column_names

# Add the rows to the table
for row in results:
    table.add_row(row)

# Print the table
print(table)

| Class | Count  |
|   0   | 284315 |
|   1   |  492   |


Show the average transaction amount for each class

In [37]:
# Create a PrettyTable object and set the style
table = pt.PrettyTable()
table.set_style(pt.MSWORD_FRIENDLY)

results = cur.execute("SELECT Class, AVG(Amount) AS 'Average Amount' FROM CREDIT_FRAUD_TABLE GROUP BY Class").fetchall()

# Get the column names from the cursor
column_names = [description[0] for description in cur.description]

# Add the column names to the table
table.field_names = column_names

# Add the rows to the table
for row in results:
    table.add_row(row)

# Print the table
print(table)

| Class |   Average Amount   |
|   0   | 88.29102242231328  |
|   1   | 122.21132113821139 |


Find the maximum transaction amount for each class

In [38]:
# Create a PrettyTable object and set the style
table = pt.PrettyTable()
table.set_style(pt.MSWORD_FRIENDLY)

results = cur.execute("SELECT Class, MAX(Amount) AS 'Maximum Amount' FROM CREDIT_FRAUD_TABLE GROUP BY Class").fetchall()

# Get the column names from the cursor
column_names = [description[0] for description in cur.description]

# Add the column names to the table
table.field_names = column_names

# Add the rows to the table
for row in results:
    table.add_row(row)

# Print the table
print(table)

| Class | Maximum Amount |
|   0   |    25691.16    |
|   1   |    2125.87     |


The following query retrive the most common amount in the transactions

In [36]:
# Create a PrettyTable object and set the style
table = pt.PrettyTable()
table.set_style(pt.MSWORD_FRIENDLY)

results = cur.execute("SELECT Amount, COUNT(*) AS Frequency FROM CREDIT_FRAUD_TABLE GROUP BY Amount ORDER BY Frequency DESC LIMIT 10").fetchall()

# Get the column names from the cursor
column_names = [description[0] for description in cur.description]

# Add the column names to the table
table.field_names = column_names

# Add the rows to the table
for row in results:
    table.add_row(row)

# Print the table
print(table)

| Amount | Frequency |
|  1.0   |   13688   |
|  1.98  |    6044   |
|  0.89  |    4872   |
|  9.99  |    4747   |
|  15.0  |    3280   |
|  0.76  |    2998   |
|  10.0  |    2950   |
|  1.29  |    2892   |
|  1.79  |    2623   |
|  0.99  |    2304   |


The following query retrieves the top 10 time periods with the highest total transaction amounts from the CREDIT_FRAUD_TABLE

In [34]:
# Create a PrettyTable object and set the style
table = pt.PrettyTable()
table.set_style(pt.MSWORD_FRIENDLY)

results = cur.execute("SELECT Time, sum(Amount) AS 'Total Amount' FROM CREDIT_FRAUD_TABLE GROUP BY Time ORDER BY sum(Amount) DESC LIMIT 10").fetchall()

# Get the column names from the cursor
column_names = [description[0] for description in cur.description]

# Add the column names to the table
table.field_names = column_names

# Add the rows to the table
for row in results:
    table.add_row(row)

# Print the table
print(table)

|   Time   |    Total Amount    |
| 166198.0 |      25691.16      |
| 48401.0  | 19682.289999999997 |
| 95286.0  |      18910.99      |
| 42951.0  |      13745.93      |
| 119713.0 |      12028.05      |
| 46253.0  |      11924.09      |
| 172273.0 |      10228.28      |
| 145283.0 |      10039.95      |
| 68780.0  |       9427.6       |
| 55709.0  |      9275.49       |
